## Building a Retrieval-Augmented Generation (RAG) System with LangChain

### Introduction

In this notebook, we will learn how to build a Retrieval-Augmented Generation (RAG) system using LangChain in Python. RAG systems combine information retrieval and natural language generation to produce answers that are grounded in external knowledge bases. This approach is particularly useful when dealing with large documents or datasets where direct querying isn’t efficient or possible.

### Objectives

- Understand the concept of Retrieval-Augmented Generation (RAG).
- Learn how to use LangChain to implement a RAG system.
- Implement the system step by step with guided TODO tasks.
- Test your implementation at each step.
- Provide helpful explanations and definitions.

Help

### Methods Used:

- LangChain: A library for building language model applications.
- VectorStore (FAISS): A tool for efficient similarity search and clustering of dense vectors.
- OpenAI Embeddings: Representations of text that can capture semantic meaning.
- RetrievalQA Chain: Combines retrieval and question-answering over documents.

### Data Used

- I extracted some chapters of the Gen AI course as a txt file.
- The goal how this notebook is to build a RAG system that can answer questions based on the content of these chapters.

## Step 1: Set Up Your Environment

We need to import the required modules and set up the OpenAI API key.

In [53]:
# Import necessary libraries
import sys
from dotenv import load_dotenv
from langchain import hub
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents.base import Document
from langchain_core.prompts import ChatPromptTemplate
from typing import List

In [54]:
load_dotenv(dotenv_path='.env.template')
sys.path.append("../")

In [55]:
from dotenv import load_dotenv
import os

# Charger les variables d'environnement
success = load_dotenv(dotenv_path='.env.template')

if success:
    print("Variables d'environnement chargées avec succès.")
else:
    print("Erreur lors du chargement des variables d'environnement.")


Variables d'environnement chargées avec succès.


In [56]:
from dotenv import load_dotenv

# Charger les variables d'environnement
load_dotenv(dotenv_path='.env.template')

# Vérifiez si les variables sont chargées correctement
import os
google_key = os.getenv("GOOGLE_API_KEY")

if google_key:
    print("Clé API Google chargée avec succès.")
else:
    print("Erreur: Clé API Google non chargée.")
    # Afficher toutes les variables d'environnement pour déboguer
    print("Variables d'environnement actuelles:", os.environ)

Clé API Google chargée avec succès.


## Step 2: Load and Split Documents

Load the document you want to use and split it into manageable chunks.

In [60]:
# Affiche le répertoire de travail actuel
import os
print(os.getcwd())  # Affiche le répertoire de travail actuel

C:\Users\USER\Desktop\GENERATIVE AI DAUPHINE\GenAI-Dauphine-Course


In [61]:
filename="C:/Users/USER/Desktop/GENERATIVE AI DAUPHINE/GenAI-Dauphine-Course"

In [62]:
if os.path.exists(filename):
    print("Le fichier existe.")
else:
    print("Le fichier n'existe pas.")


Le fichier existe.


In [66]:
# Load your document and split it into chunks
# Hint: Use TextLoader and RecursiveCharacterTextSplitter


# Specify the filename
filename = "C:/Users/USER/Desktop/GENERATIVE AI DAUPHINE/GenAI-Dauphine-Course/data/gen_ai_course.txt"
#"C:/Users/USER/Desktop/GENERATIVE AI DAUPHINE/GenAI-Dauphine-Course/data/gen_ai_course.txt"

# Load the document
loader = TextLoader(filename)
documents = loader.load()

# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

RuntimeError: Error loading C:/Users/USER/Desktop/GENERATIVE AI DAUPHINE/GenAI-Dauphine-Course/data/gen_ai_course.txt

## Step 3: Create Embeddings and Build the VectorStore

Generate embeddings for each chunk and store them in a vector store for efficient retrieval.

In [ ]:
# Create embeddings and store them in a VectorStore
# Hint: Use OpenAIEmbeddings and FAISS
embeddings = ...
vectorstore = ...
# Hint : Use GoogleGenerativeAIEmbeddings(model=...)

## Step 4: Set Up the QA Chain using LCEL

Create a chain that can retrieve relevant chunks and generate answers based on them.

In [ ]:
llm = ...  # Initialize ChatGoogleGenerativeAI with the required arguments

#Create a function to format documents for the prompt
def format_docs(docs: List[Document]):
    # Hint: Join the content of each document
    return ...  # Join the page content of docs into a stringg

# Hint: Define the prompt template with system and human messages. See help below
prompt = ...

# Hint: Format the documents using the function above
formatted_docs = ...

# Hint: Create the QA chain by combining the prompt and model
qa_chain = ...


## Step 5: Ask Questions and Get Answers

Test the system by asking a question.

In [ ]:
# Ask a question to the QA chain
# Replace 'Your question here' with an actual question and run the qa_chain for this question

# Answer:
query = "What is the main topic discussed in the document?"
result = ...
print(result)

Ellipsis


## Step 6: Test Your Implementation with Different Questions

Try out different questions to see how the system performs.

In [ ]:
# Replace 'Another question here' with your own question and run the qa_chain for this question

query = "Can you summarize the key points mentioned?"
result = ...
print(result)

Ellipsis


## Step 7: Improve the System

You can experiment with different parameters, like adjusting the chunk size or using a different language model.

Conclusion

Congratulations! You’ve built a simple Retrieval-Augmented Generation system using LangChain. This system can retrieve relevant information from documents and generate answers to user queries.

Help

- TextLoader: Loads text data from files.
- RecursiveCharacterTextSplitter: Splits text into smaller chunks for better processing.
- FAISS: A library for efficient similarity search of embeddings.
- RetrievalQA Chain: A chain that retrieves relevant documents and answers questions based on them.
- OpenAIEmbeddings: Generates embeddings that capture the semantic meaning of text.

## Help

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

prompt_value = template.invoke(
    {
        "name": "Bob",
        "user_input": "What is your name?"
    }
)

# Output:
# ChatPromptValue(
#    messages=[
#        SystemMessage(content='You are a helpful AI bot. Your name is Bob.'),
#        HumanMessage(content='Hello, how are you doing?'),
#        AIMessage(content="I'm doing well, thanks!"),
#        HumanMessage(content='What is your name?')
#    ]
#)

messages=[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})]
